In [1]:
import pathlib
import logging
import threading
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import seaborn as sns
tfd = tfp.distributions
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

print(tf.__version__)
dispositivoUsado = '/cpu:0' 
#dispositivoUsado = '/gpu:0'
exibirRodando = False
balancearDados = False
balancearPor = 'QoE'
QuantasThreadsFinalizaram = 0
anonimo = False

2.7.0


In [2]:
def balancearDadosPor(dados,por):
    g = dados.groupby(por)
    g = g.apply(lambda x: x.sample(g.size().min())).reset_index(drop=True)
    g = g.sample(frac=1).reset_index(drop=True)
    return g

In [3]:
dados = pd.read_csv('dados-Ambos-Tudo.csv')
dados["QoE"] = dados["QoE"] + 3
plt.rcParams['figure.dpi'] = 600
print("------------------------------------------------------")
# dados["Jogo"] = dados["Jogo"].apply(lambda jogo: 0 if jogo == "Spelunky" else 1)
print("Maior valor de uso de CPU:RAM:GPU:VRAM")
print(dados['UsoCpu'].max())
print(dados['UsoRam'].max())
print(dados['UsoGpu'].max())
print(dados['UsoVram'].max())
print("CPU acima de 80%")
print(dados[dados.UsoCpu > 80])
print("RAM acima de 80%")
print(dados[dados.UsoRam > 80])

#---------------------------------------------------------
#DROPANDO LINHAS COM USO DE CPU E RAM ALTO

dados = dados[dados['UsoCpu'] < 81]
dados = dados[dados['UsoRam'] < 81]
dados = dados[dados['Jogo'] != 'FEAR']
dados.sample(frac=1).reset_index(drop=True)

#DROPANDO LINHAS COM USO DE CPU E RAM ALTO
#---------------------------------------------------------

if(anonimo):
    nomes = dados.NOME.unique()
    player = 1
    for i in range(len(nomes)):
        dados['NOME'] = dados['NOME'].str.replace(nomes[i],"Jogador "+str(player))
        player = player + 1

if(balancearDados):
    dados = balancearDadosPor(dados,balancearPor)
else:
    dados = dados.sample(frac=1).reset_index(drop=True)
    
porcentagemTeste = 0.2
porcentagemValidacao = 0.1

#Salvando quantos jogadores temos e quantas instancias temos

print("-------------------Salvando-------------------------")
numeroInstancias = len(dados)
dicionarioJogadores = dados['NOME'].value_counts().to_dict()
print("-------------------Salvando-------------------------")

dados.drop(['TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram', 'TerminoPartida','Jogo','NOME'], axis=1, inplace=True)
dados.dropna(inplace=True)

porcentagemTeste = 0.2
porcentagemValidacao = 0.1
NumeroCamadasInternas = 32
metodoAtivacao = ['relu','linear','softmax','softplus','softsign','tanh','selu','gelu','elu']
AtivacaoAtual = metodoAtivacao[4]

------------------------------------------------------
Maior valor de uso de CPU:RAM:GPU:VRAM
99
84
41
35
CPU acima de 80%
         NOME  QoE  TempoJogo  UsoCpu  UsoRam  UsoGpu  UsoVram  DelayVideo  \
12   Jogador1    4          3      86      45      22       19          50   
13   Jogador1    1          3      87      45      22       19         100   
14   Jogador1    0          3      85      45      22       19         125   
15   Jogador1    2          3      86      45      22       19           0   
16   Jogador1    4          3      86      45      21       18         100   
..        ...  ...        ...     ...     ...     ...      ...         ...   
562  Jogador1    4          3      96      40      21       18           0   
715  Jogador1    0          3      81      37      31       27           0   
718  Jogador1    1          3      87      37      22       19         125   
731  Jogador1    6          3      81      38      21       19           0   
916  Jogador1    4 

In [4]:
def build_model_1(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [5]:
def build_model_2(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [6]:
def build_model_3(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [7]:
def build_model_4(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [8]:
def build_model_5(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [9]:
def build_model_6(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [10]:
def build_model_7(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [11]:
def build_model_8(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [12]:
def build_model_9(X_train):
 with tf.device(dispositivoUsado):
  model = keras.Sequential([
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual, input_shape=[X_train.shape[1]]),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(NumeroCamadasInternas, activation=AtivacaoAtual),
    layers.Dense(1)
  ])
    
  optimizer = 'adam'

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [13]:
def arredondaVetor(vetor):
 with tf.device(dispositivoUsado):
    novoVetor = []
    for x in range(len(vetor)):
        if(vetor[x]>=0):
            floatTeste = vetor[x] - int(vetor[x])
            if(floatTeste>=0.5):
                novoVetor.append(int(vetor[x])+1)
            else:
                novoVetor.append(int(vetor[x]))
        else:
            valor = vetor[x]*(-1.0)
            floatTeste = valor - int(valor)
            if(floatTeste>=0.5):
                novoVetor.append((int(valor)+1)*-1)
            else:
                novoVetor.append(int(valor)*-1)
    return novoVetor

In [14]:
def porcentagemAcerto(vetor1,vetor2):
  with tf.device(dispositivoUsado):
    tamanho = float(len(vetor1))
    acertos = float(0)
    for x in range(len(vetor1)):
        if(vetor1[x]==vetor2[x]):
            acertos = acertos + 1.0
    acertos = (acertos/tamanho)*100.0
    return acertos

In [15]:
def rodarModelo(numero,baseDados):
 with tf.device(dispositivoUsado):
    if 'NOME' in baseDados.columns:
        baseDados.drop(['NOME'], axis=1, inplace=True)
        baseDados.dropna(inplace=True)
        
    global porcentagemTeste
    global porcentagemValidacao
    
    X = np.asarray(baseDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 'PerdaVideo', 'PerdaComandos']])
    y = np.asarray(baseDados['QoE'])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_temp = np.zeros((baseDados.shape[0],baseDados.shape[1]))
    X_temp[:,:X.shape[1]] = X
    X = X_temp
    X[:, 6:-1] = np.asarray(baseDados[list(baseDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo','JitterComandos', 'PerdaVideo', 'PerdaComandos', 'QoE'], axis=1))])
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    

    class PrintDot(keras.callbacks.Callback):
      #def on_epoch_end(self, epoch, logs):
      #  if epoch % 100 == 0: print('')
      #  print('.', end='')
        def on_train_end(self, logs=None):
            global QuantasThreadsFinalizaram
            QuantasThreadsFinalizaram = QuantasThreadsFinalizaram + 1
            print(QuantasThreadsFinalizaram,end=',')


    EPOCHS = 1000
    
    verbose = 0
    if(exibirRodando):
        verbose = 1
    
    
    modelos = list()
    mses = list()
    losss = list()
    maes = list()
    acertos = list()
    historys = list()
    kFold = StratifiedKFold(n_splits=10)
    #start = time.process_time()#Contar tempo de execução
    for train, test in kFold.split(X_train, y_train):

        model = build_model_1(X_train[train])
        if(numero == 1):
            model = build_model_1(X_train[train])
        elif(numero == 2):
            model = build_model_2(X_train[train])
        elif(numero == 3):
            model = build_model_3(X_train[train])
        elif(numero == 4):
            model = build_model_4(X_train[train])
        elif(numero == 5):
            model = build_model_5(X_train[train])
        elif(numero == 6):
            model = build_model_6(X_train[train])
        elif(numero == 7):
            model = build_model_7(X_train[train])
        elif(numero == 8):
            model = build_model_8(X_train[train])
        elif(numero == 9):
            model = build_model_9(X_train[train])
            
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,verbose=verbose)
    
        history = model.fit(X_train[train], y_train[train], epochs=EPOCHS,
                        validation_split = porcentagemValidacao, verbose=verbose, callbacks=[early_stop])
        loss, mae, mse = model.evaluate(X_train[test], y_train[test], verbose=verbose)
        
        test_predictions = model.predict(X_train[test],verbose).flatten()
        test_predictions_round = arredondaVetor(test_predictions)
        porcentagemAcertos = porcentagemAcerto(y_train[test],test_predictions_round)
        mses.append(mse)
        modelos.append(model)
        losss.append(loss)
        maes.append(mae)
        historys.append(history)
        acertos.append(porcentagemAcertos)
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    mae = maes[iMSE]
    loss = losss[iMSE]
    history = historys[iMSE]
    porcentagemAcertos = acertos[iMSE]
    #global QuantasThreadsFinalizaram
    #print(str(QuantasThreadsFinalizaram)+": MSE: "+str(round(mse,4))+" tempo: "+str(round(time.process_time() - start,2)))
    #QuantasThreadsFinalizaram = QuantasThreadsFinalizaram + 1
    
    if(exibirRodando):
        print("Conjunto de testes Mean Abs Error: {:5.2f} QoE".format(mae))
    
    loss1, mae1, mse1 = model.evaluate(X_test, y_test, verbose=verbose)
    
    test_predictions = model.predict(X_test,verbose).flatten()
    
    test_predictions_round = arredondaVetor(test_predictions)
    porcentagemAcertos1 = porcentagemAcerto(y_test,test_predictions_round)
    
    if(mse1<mse):
        mse = mse1
        loss = loss1
        mae = mae1
        porcentagemAcertos = porcentagemAcertos1
        
    if(exibirRodando):
        plt.scatter(y_test, test_predictions)
        #Fazer um MSE e MAE destes dados, para ter também da predição.
        plt.xlabel('Valores Reais [QoE]')
        plt.ylabel('Predicões [QoE]')
        plt.axis('equal')
        plt.axis('square')
        plt.xlim([0,plt.xlim()[1]])
        plt.ylim([0,plt.ylim()[1]])
        _ = plt.plot([-100, 100], [-100, 100])
        plt.show()
        
        print("Porcentagem de acertos arredondado: "+str(porcentagemAcertos))
    
        plt.figure()
        plt.scatter(y_test, test_predictions_round)
        plt.xlabel('Valores Reais [QoE]')
        plt.ylabel('Predicões Arredondadas [QoE]')
        plt.axis('equal')
        plt.axis('square')
        plt.xlim([0,plt.xlim()[1]])
        plt.ylim([0,plt.ylim()[1]])
        _ = plt.plot([-100, 100], [-100, 100])
        plt.show()
    
    resultado = []
    resultado.append(loss)
    resultado.append(mae)
    resultado.append(mse)
    resultado.append(porcentagemAcertos)
    return resultado,y_test,test_predictions,history

In [16]:
def plot_heatMap(test_predictions,y_test,nomeHeatmap):
    import seaborn as sns
    test_predictions_round = arredondaVetor(test_predictions)
    x_axis_labels = ['E. Péssimo','Péssimo','Ruim','Regular','bom','Excelente','Ideal'] # labels for x-axis
    y_axis_labels = ['E. Péssimo','Péssimo','Ruim','Regular','bom','Excelente','Ideal'] # labels for y-axis
    
    heatMatrix = np.zeros((7,7))
    for x in range(len(y_test)):
        intX = int(test_predictions_round[x])
        intY = int(y_test[x])
        if(intX > 6):
            continue
        heatMatrix[intX][intY] = heatMatrix[intX][intY] +1
                
    sns.heatmap(heatMatrix, xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="Greys")
    plt.savefig(nomeHeatmap+'.png', transparent=False)
    plt.show()

In [17]:
def plot_melhor(y_test, test_predictions,history,tipo):
 with tf.device(dispositivoUsado):
    histi = pd.DataFrame(history.history)
    histi['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro Médio Absoluto [QoE]')
    plt.plot(histi['epoch'], histi['mae'],label='Erro de Treino')
    plt.plot(histi['epoch'], histi['val_mae'],label = 'Erro de Validação')
    plt.ylim([0,5])
    plt.legend()
    plt.savefig(tipo+'-SoRedeMAE.png')
    plt.show()
    
        
    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro Médio Quadratico [$QoE^2$]')
    plt.plot(histi['epoch'], histi['mse'],label='Erro de Treino')
    plt.plot(histi['epoch'], histi['val_mse'],label = 'Erro de Validação')
    plt.ylim([0,20])
    plt.legend()
    plt.savefig(tipo+'-SoRedeMSE.png')
    plt.show()
    
    
    plt.figure()
    plt.scatter(y_test, test_predictions)
    plt.xlabel('Valores Reais [QoE]')
    plt.ylabel('Predicões [QoE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig(tipo+'-SoRedePredicoesSemArredondar.png')
    plt.show()
    
    
    
    
    test_predictions_round = arredondaVetor(test_predictions)
    print("Porcentagem de acertos arredondado: "+str(porcentagemAcerto(y_test,test_predictions_round)))
    plt.figure()
    plt.scatter(y_test, test_predictions_round)
    plt.xlabel('Valores Reais [QoE]')
    plt.ylabel('Predicões Arredondadas [QoE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig(tipo+'-SoRedePredicoesArredondado.png')
    plt.show()
    
    
    plot_heatMap(test_predictions,y_test,"HeatMap_"+tipo)

In [18]:
def salvaLinhaEmArquivo(linha):
    with open('melhoresResultados.txt', 'a') as f:
        f.writelines(linha)
        f.close()

In [19]:
global porcentagemTeste
global porcentagemValidacao
global NumeroCamadasInternas
global metodoAtivacao
global AtivacaoAtual

umMetodo = False
repeticoes = 5
porcentagemTeste = 0.2
porcentagemValidacao = 0.1

AGeralMse = 'softsign'
BGeralMse = 1
CGeralMse = 32
AGeralAcerto = 'softsign'
BGeralAcerto = 1
CGeralAcerto = 32

saida,y,predicao,hist = rodarModelo(8,dados)
melhorAcerto = saida[3]
melhorAcertoMSE = saida[2]
melhorMSE = saida[2]
melhorMSEAcerto = saida[3]
yMelhorMSE = y
yMelhorAcerto = y
predicaoMelhorMSE = predicao
predicaoMelhorAcerto = predicao
histMelhorMSE = hist
histMelhorAcerto = hist
qualModeloMSE = 'softsign'
qualModeloAcerto = 'softsign'
melhorCamadasMSE = NumeroCamadasInternas
melhorCamadasAcerto = NumeroCamadasInternas

if(umMetodo):
    with tf.device(dispositivoUsado):
        for B in range(9):#8
            for C in range(4,129,4):#129
              for ABA in range(repeticoes):
                if(exibirRodando):
                    print("------------------------------------------------------------------")
                    print("Ativação atual: "+str(AGeralMse))
                    print("Quantidade de camadas internas: "+str(B+1))
                    print("Numero neuronios camadas internas: "+str(C))
                NumeroCamadasInternas = C
                AtivacaoAtual = AGeralMse
                saida,y,predicao,hist = rodarModelo(B+1,dados)
                
                if(saida[3]>melhorAcerto):
                    melhorAcerto = saida[3]
                    melhorAcertoMSE = saida[2]
                    yMelhorAcerto = y
                    predicaoMelhorAcerto = predicao
                    histMelhorAcerto = hist
                    BGeralAcerto = B+1
                    CGeralAcerto = C
                if(saida[2]<melhorMSE):
                    melhorMSEAcerto = saida[3]
                    melhorMSE = saida[2]
                    yMelhorMSE = y
                    predicaoMelhorMSE = predicao
                    histMelhorMSE = hist
                    BGeralMse = B+1
                    CGeralMse = C
                if(exibirRodando):
                    print("O resultado MSE foi: "+str(saida[2])+" com "+str(saida[3])+"% de acerto")
                    print("Com os seguintes parametros:")
                    print("Ativação atual: "+str(AGeral))
                    print("Quantidade de camadas internas: "+str(B+1))
                    print("Numero neuronios camadas internas: "+str(C))
                    print("------------------------------------------------------------------")
                    print("------------------------------------------------------------------")
        print("------------------------------------------------------------------")
        print("------------------------------------------------------------------")
        saidaTexto = "O melhor resultado MSE foi " + str(melhorMSE) + " com "+str(melhorMSEAcerto)+"% de acerto para os seguintes parâmetros:\n"\
            + "Ativação: "+str(AGeralMse) + "\nQuantidade de camadas internas: "+str(BGeralMse) + "\nNúmero neurônios camadas internas: "+str(CGeralMse)\
               + "------------------------------------------------------------------\n------------------------------------------------------------------\n"\
                "O melhor resultado de acerto foi " + str(melhorAcerto) + "% com MSE "+str(melhorAcertoMSE)+"% de acerto para os seguintes parâmetros:\n"\
            + "Ativação: "+str(AGeralAcerto) + "\nQuantidade de camadas internas: "+str(BGeralAcerto) + "\nNúmero neurônios camadas internas: "+str(CGeralAcerto)\
               + "------------------------------------------------------------------\n------------------------------------------------------------------\n"
        print(saidaTexto)
        f = open("DescobrirMelhoresParametrosSemAcordo.txt","w+")
        f.write(saidaTexto)
        f.close()

else:
  valorRepeticao = 1
  #Colocar os melhores valores da hora que parou aqui, para continar depois
  with tf.device(dispositivoUsado):
    for A in metodoAtivacao:
        for B in range(9):#8
            for C in range(16,129,16):#129
              print(valorRepeticao)
              valorRepeticao = valorRepeticao + 1
              for ABA in range(repeticoes):
                if(exibirRodando):
                    print("------------------------------------------------------------------")
                    print("Ativação atual: "+str(A))
                    print("Quantidade de camadas internas: "+str(B+1))
                    print("Numero neuronios camadas internas: "+str(C))
                NumeroCamadasInternas = C
                AtivacaoAtual = A
                saida,y,predicao,hist = rodarModelo(B+1,dados)
                
                if(saida[3]>melhorAcerto):
                    melhorAcerto = saida[3]
                    melhorAcertoMSE = saida[2]
                    yMelhorAcerto = y
                    predicaoMelhorAcerto = predicao
                    histMelhorAcerto = hist
                    qualModeloAcerto = A
                    AGeralAcerto = A
                    BGeralAcerto = B+1
                    CGeralAcerto = C
                if(saida[2]<melhorMSE):
                    melhorMSEAcerto = saida[3]
                    melhorMSE = saida[2]
                    yMelhorMSE = y
                    predicaoMelhorMSE = predicao
                    histMelhorMSE = hist
                    qualModeloMSE = A
                    AGeralMse = A
                    BGeralMse = B+1
                    CGeralMse = C
                    s = "\n------------------------------------------------------------------------------\n"\
                     +"Melhor MSE até agora: "+str(round(melhorMSE,4))+" com acerto: "+str(round(melhorMSEAcerto,4))\
                     +"\n Modelo: "+str(AGeralMse)+" N. camadas internas: "+str(BGeralMse)+" com "+str(CGeralMse)+" neurônios."
                    salvaLinhaEmArquivo(s)
                    print(s)
                if(exibirRodando):
                    print("O resultado MSE foi: "+str(saida[2])+" com "+str(saida[3])+"% de acerto")
                    print("Com os seguintes parametros:")
                    print("Ativação atual: "+str(A))
                    print("Quantidade de camadas internas: "+str(B+1))
                    print("Numero neuronios camadas internas: "+str(C))
                    print("------------------------------------------------------------------")
                    print("------------------------------------------------------------------")
    print("------------------------------------------------------------------")
    print("------------------------------------------------------------------")
    saidaTexto = "O melhor resultado MSE foi " + str(melhorMSE) + " com "+str(melhorMSEAcerto)+"% de acerto para os seguintes parâmetros:\n"\
        + "Ativação: "+str(AGeralMse) + "\nQuantidade de camadas internas: "+str(BGeralMse) + "\nNúmero neurônios camadas internas: "+str(CGeralMse)\
        + "\n------------------------------------------------------------------\n------------------------------------------------------------------\n"\
        "O melhor resultado de acerto foi " + str(melhorAcerto) + "% com MSE "+str(melhorAcertoMSE)+"% de acerto para os seguintes parâmetros:\n"\
        + "Ativação: "+str(AGeralAcerto) + "\nQuantidade de camadas internas: "+str(BGeralAcerto) + "\nNúmero neurônios camadas internas: "+str(CGeralAcerto)\
        + "\n------------------------------------------------------------------\n------------------------------------------------------------------\n"
    print(saidaTexto)
    f = open("DescobrirMelhoresParametrosSemAcordo.txt","w+")
    f.write(saidaTexto)
    f.close()

KeyboardInterrupt: 

In [ ]:
print("------------Melhor Acerto----------------")
print("MSE: "+str(melhorAcertoMSE))
print("Acerto: "+str(melhorAcerto))
print("Met. Ativação: "+str(AGeralAcerto))
print("N. Camadas: "+str(BGeralAcerto))
print("N. de neuronios: "+str(CGeralAcerto))
print("------------Melhor MSE----------------")
print("MSE: "+str(melhorMSE))
print("Acerto: "+str(melhorMSEAcerto))
print("Met. Ativação: "+str(AGeralMse))
print("N. Camadas: "+str(BGeralMse))
print("N. de neuronios: "+str(CGeralMse))
salvaLinhaEmArquivo("\n----------------------------------------------------------------------------------------\n")
salvaLinhaEmArquivo(saidaTexto)

In [ ]:
print(saidaTexto)
print("Plot do melhor MSE:")
plot_melhor(yMelhorMSE,predicaoMelhorMSE,histMelhorMSE,"Achar-Melhores-Parametros-MSE")
print("Plot do melhor Acerto:")
plot_melhor(yMelhorMSE,predicaoMelhorMSE,histMelhorMSE,"Achar-Melhores-Parametros-Acerto")